In [7]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, BatchNormalization, Dropout
# from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

In [8]:
# lectura de los datos
(val_X, val_Y), (train_X, train_Y) = mnist.load_data()

In [9]:
# pre-procesamiento de los datos

train_X = train_X.reshape(train_X.shape[0],28,28,1)
val_X = val_X.reshape(val_X.shape[0],28,28,1)
train_X = train_X.astype('float32')
val_X = val_X.astype('float32')

# normalizar datos de [0..255] a [0..1]
train_X /= 255
val_X /= 255

# convierte etiqueta de clase en "one-hot vectors" [0,0,1,0,...]
num_classes = 10
train_Y = keras.utils.to_categorical(train_Y, num_classes)
val_Y = keras.utils.to_categorical(val_Y, num_classes)

**Efectos visibles de técnicas de regularización**
* BatchNormalization + MaxPolling(2,2) + Dropout(0.25) tras Flatten y Dense(100)

In [10]:
# modelo: capa entrada + 20 conv 5x5 + 40 conv 5x5 + densa(100) + capa densa salida (10)

model = Sequential()

model.add(Conv2D(20, (5, 5), input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(40, (5, 5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

# imprime resumen del modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 20)        520       
                                                                 
 batch_normalization_2 (Bat  (None, 24, 24, 20)        80        
 chNormalization)                                                
                                                                 
 activation_2 (Activation)   (None, 24, 24, 20)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 12, 12, 20)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 40)          20040     
                                                                 
 batch_normalization_3 (Bat  (None, 8, 8, 40)         

In [11]:
# hiperparámetros del proceso de entrenamiento
batch_size = 100
epochs = 20

# optimizador (actualización de pesos)
opt=SGD(learning_rate=0.05, momentum=0.9)

# configuración del proceso de entrenamiento
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# entrenamiento del modelo
history = model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(val_X, val_Y))

Epoch 1/20
100/100 [==============================] - 7s 38ms/step - loss: 0.3923 - accuracy: 0.8771 - val_loss: 0.1777 - val_accuracy: 0.9467
Epoch 2/20
100/100 [==============================] - 3s 31ms/step - loss: 0.1266 - accuracy: 0.9596 - val_loss: 0.1353 - val_accuracy: 0.9599
Epoch 3/20
100/100 [==============================] - 2s 21ms/step - loss: 0.0857 - accuracy: 0.9713 - val_loss: 0.1301 - val_accuracy: 0.9602
Epoch 4/20
100/100 [==============================] - 2s 23ms/step - loss: 0.0687 - accuracy: 0.9772 - val_loss: 0.0962 - val_accuracy: 0.9714
Epoch 5/20
100/100 [==============================] - 2s 23ms/step - loss: 0.0539 - accuracy: 0.9822 - val_loss: 0.1279 - val_accuracy: 0.9632
Epoch 6/20
100/100 [==============================] - 3s 31ms/step - loss: 0.0457 - accuracy: 0.9848 - val_loss: 0.0663 - val_accuracy: 0.9797
Epoch 7/20
100/100 [==============================] - 3s 31ms/step - loss: 0.0413 - accuracy: 0.9856 - val_loss: 0.0858 - val_accuracy: 0.9747

In [12]:
# resultados sobre entrenamiento
train_results = model.evaluate(train_X, train_Y, verbose=0)
print('Error (loss) sobre datos de entrenamiento: {:.4f}'.format(train_results[0]))
print('Tasa de éxito de clasificación (accuracy) sobre datos de entrenamiento: {:.4f}\n'.format(train_results[1]))

# resultados sobre validación
val_results = model.evaluate(val_X, val_Y, verbose=0)
print('Error (loss) sobre datos de VALIDACIÓN: {0:.4f}'.format(val_results[0]))
print('Tasa de éxito de clasificación (accuracy) sobre datos de VALIDACIÓN: {0:.4f}\n'.format(val_results[1]))

Error (loss) sobre datos de entrenamiento: 0.0013
Tasa de éxito de clasificación (accuracy) sobre datos de entrenamiento: 0.9998

Error (loss) sobre datos de VALIDACIÓN: 0.0620
Tasa de éxito de clasificación (accuracy) sobre datos de VALIDACIÓN: 0.9843

